# LangChain과 Chroma를 활용한 RAG 구성

1. 문서의 내용을 읽는다
2. 문서를 쪼갠다
    - 토큰수 초과로 답변을 생성하지 못할 수도 있고
    - 문서가 길면 (인풋이 길면) 답변 생성이 오래걸림
    - split 된 데이터 chunk를 Large Language Model(LLM)에게 전달하면 토큰 절약 가능
    - 비용 감소와 답변 생성시간 감소의 효과
3. 임베딩 --> 벡터 DB에 저장
4. 질문이 있을 때, 벡터 DB에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

## 1. 문서 쪼개서 document_list 생성
- 마크다운 파일 1개를 1개의 document 객체로 만들어 리스트 반환 (문서 쪼개는 것 대신)

In [3]:
import os
import json
import re
from langchain.schema import Document # langchain_core.documents 로 변경 가능

def load_documents_from_metadata_json(
    md_dir="md_pdf",
    metadata_filepath="all_metadata.json"
):
    """
    all_metadata.json 파일을 기반으로 문서를 생성합니다.
    JSON 안의 각 메타데이터에 해당하는 md 파일의 내용을 읽어와
    images 정보를 포함한 전체 메타데이터와 함께 Document 객체를 만듭니다.
    """
    # 1. 먼저 all_metadata.json 파일을 읽어옵니다.
    with open(metadata_filepath, "r", encoding="utf-8") as f:
        all_metadata = json.load(f)

    document_list = []
    # 2. 파일 목록이 아닌, JSON 안의 메타데이터 목록을 순회합니다.
    for metadata in all_metadata:
        md_file_name = metadata.get("source")
        if not md_file_name:
            continue

        file_path = os.path.join(md_dir, md_file_name)
        try:
            with open(file_path, "r", encoding="utf-8") as f:
                content = f.read()

            # 3. 파일 내용과 함께, JSON에서 가져온 '완전한' 메타데이터를 사용합니다.
            document_list.append(
                Document(
                    page_content=content,
                    metadata=metadata  # images 정보가 포함된 전체 메타데이터
                )
            )
        except FileNotFoundError:
            print(f"⚠️  경고: {file_path} 파일을 찾을 수 없습니다. 건너뜁니다.")
            continue

    print(f"✅ Document 개수: {len(document_list)}")
    return document_list


# ===== 실행 예시 =====
if __name__ == "__main__":
    # 새로운 함수를 호출합니다.
    document_list = load_documents_from_metadata_json(
        md_dir="md_pdf",
        metadata_filepath="all_metadata.json"
    )

✅ Document 개수: 49


In [11]:
document_list[1].metadata

{'source': '002.md',
 'origin_pdf': '무역관정산교육_v6_0923.pdf',
 'page_num': 2,
 'images': []}

In [12]:
document_list[3].metadata

{'source': '004.md',
 'origin_pdf': '무역관정산교육_v6_0923.pdf',
 'page_num': 4,
 'images': ['004_img0.png']}

In [13]:
print(document_list[3])

page_content='1.1.4.2. 카드명세서상의 잔액

  

국가마다 카드사마다 명세서양식은 천차만별 입니다. 카드사마다 청구대상기간도 다 다릅니다. 보통 당월 미결제액(=당월 청구액=카드를  
긁었으나 아직 대금결제가 이루어지지 않은 금액)이 기재가 되어있고, 과거 청구액을 모두 납부했다면 이 당월 미결제액(=당월 청구액)이  
카드 잔액이 됩니다.

이 카드명세서상의 잔액을 ERP < 자금잔액명세서> 메뉴에  
<나. 은행잔고대사표-은행잔 고증명서-신용카드금액> 컬럼에 입력을 해야 합니다.

아래 예시 명세 서를 기준으로 잔액을 계산하는 방법 입니다. 1. 마지 막 청구 액(명세 서의 잔액)은  
981.12 CNY' 입니다. 2. 이전 달 청구 액은 모두 납부 완료 했다고 가정 합니다. (즉, 과거 미납액  
없음) 3. 따라서, 이번 달에 납부해야 할 최종 카드 대금명 세서상 잔액은 \*\*981.12 CNY\*\*가 됩니다.

· 잔액증명 예시 : 카드대금명세서' metadata={'source': '004.md', 'origin_pdf': '무역관정산교육_v6_0923.pdf', 'page_num': 4, 'images': ['004_img0.png']}


## 2. 임베딩

In [14]:
# 3. 임베딩 해주기 --> 백터DB에 저장 (Chroma 쓸꺼고 / 인메모리라 간단)
from dotenv import load_dotenv # 임베딩에 openai key 인자가 있어서 환경변수를 로드해줘야 함
from langchain_openai import OpenAIEmbeddings 

# 환경변수 불러옴
load_dotenv()

# OpenAI에서 제공하는 Embedding Model을 활용해서 `chunk`를 vector화
embedding = OpenAIEmbeddings(model = 'text-embedding-3-large') # 임베딩 모델을 large로 바꿔주기

## 3. 벡터DB 생성 (Pinecone) 

In [ ]:
'''
# Pinecone 설치
%pip install --upgrade --quiet \
    langchain-pinecone \
    langchain-openai \
    langchain
'''

In [15]:
import os

from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

index_name = 'finance-new-index'
pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)

# 데이터를 추가할 때는 `from_documents()` 데이터를 추가한 이후에는 `from_existing_index()`를 사용합니다
#database = PineconeVectorStore.from_documents(document_list, embedding, index_name=index_name)
database = PineconeVectorStore.from_existing_index(index_name=index_name, embedding=embedding)

/Users/dayeayim/.pyenv/versions/inflearn-streamlit/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 4. 문서 검색 및 답변
- 벡터DB에서 적합한 문서 찾고 LLM에 문서와 질의를 주면서 답변을 요청(RetrievalQA)
- RetrievalQA: 데이터를 검색(Retrieval)한 다음에 질문(Question)하고 답변(Answer) 할 것이다

In [16]:
# 문서를 가지고 왔으니까 질의를 해봐야 한다 
from langchain_openai import ChatOpenAI 

llm = ChatOpenAI(model = 'gpt-4o')

In [17]:
# Retrieval된 데이터는 LangChain에서 제공하는 프롬프트("rlm/rag-prompt") 사용해서 답변해보기
from langchain import hub 
from langchain.prompts import ChatPromptTemplate

# 기본 RAG 프롬프트 가져오기
base_prompt = hub.pull("rlm/rag-prompt")

# 시스템 역할 지시 추가해서 새 템플릿 만들기
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 재무팀에서 근무하는 정산 전문가입니다. "
               "[context]를 참고해서 사용자의 질문에 답변해주세요. "
               "[context]에 없는 정보는 지어내지 말고, 자료에 없음이라고 답하세요."),
    *base_prompt.messages  # 기존 RAG 메시지 구조 유지
])

In [54]:
# 답변생성 (QA 체인 만들기)
#-- RetrievalQA를 통해 LLM에 전달
#-- RetrievalQA는 create_retrieval_chain으로 대체됨
#-- 실제 ChatBot 구현 시 create_retrieval_chain으로 변경하는 과정을 볼 수 있음

from langchain.chains import RetrievalQA 

retriever = database.as_retriever( # 벡터DB
    search_kwargs={"k": 2}) # 유사도로 몇 개 문서 찾을 것인지
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever = retriever, # 위에서 만든 retriever (DB에서 문서 검색)
    chain_type_kwargs = {"prompt": prompt}, # 위에서 만든 prompt (정산 전문가)
    return_source_documents=True # 문서 소스 포함
)

In [49]:
query = 'erp에서 송금현황 조회하는 방법 알려줘'

In [50]:
ai_message = qa_chain.invoke({"query": query})

# 답변 본문
answer = ai_message["result"]

In [51]:
ai_message

{'query': 'erp에서 송금현황 조회하는 방법 알려줘',
 'result': 'ERP에서 송금현황을 조회하려면 "전도관리 – 전도금배정/정산현황 - 송금현황" 경로로 이동하면 됩니다. 해당 메뉴에서는 본사에서 송금된 내역을 송금일자별로 조회할 수 있습니다.',
 'source_documents': [Document(id='fdeb6dd5-b4d1-47c9-8d43-8fc4047d7686', metadata={'images': ['012_img0.png'], 'origin_pdf': '무역관정산교육_v6_0923.pdf', 'page_num': 12.0, 'source': '012.md'}, page_content='2.2. 송금현황\n=========\n\n  \n\n2.2.1. ERP 메뉴 경로 : 전도관리 – 전도금배정/정산현황 - 송금현황  \n2.2.2. 메뉴 설명 : 본사에서 송금된 내역을 송금일자별로 조회할수 있는 메뉴\n\n• ERP 송금현황 조회방법'),
  Document(id='9c24f7bd-af31-402b-b1b9-eaa9e1b189df', metadata={'images': ['014_img0.png'], 'origin_pdf': '무역관정산교육_v6_0923.pdf', 'page_num': 14.0, 'source': '014.md'}, page_content='2.3. 전도금배정현황\n============\n\n  \n\n2.3.1. ERP 메뉴 경로 : 전도관리 – 전도금배정/정산현황 - 전도금배정현황조회  \n2.3.2. 메뉴 설명 : 본사에서 송금된 상세내역을 기간별로 조회할수 있는 메뉴\n\n• ERP 전도금배정현황 조회방법\n=================='),
  Document(id='e3ae3298-64ee-470b-a9ca-38a6cb12a015', metadata={'images': ['010_img0.png'], 'origin_pdf': '무역관정산교육_v6_0923.pdf', 'page_num': 1

In [52]:
answer

'ERP에서 송금현황을 조회하려면 "전도관리 – 전도금배정/정산현황 - 송금현황" 경로로 이동하면 됩니다. 해당 메뉴에서는 본사에서 송금된 내역을 송금일자별로 조회할 수 있습니다.'

In [53]:
ai_message["source_documents"]

[Document(id='fdeb6dd5-b4d1-47c9-8d43-8fc4047d7686', metadata={'images': ['012_img0.png'], 'origin_pdf': '무역관정산교육_v6_0923.pdf', 'page_num': 12.0, 'source': '012.md'}, page_content='2.2. 송금현황\n=========\n\n  \n\n2.2.1. ERP 메뉴 경로 : 전도관리 – 전도금배정/정산현황 - 송금현황  \n2.2.2. 메뉴 설명 : 본사에서 송금된 내역을 송금일자별로 조회할수 있는 메뉴\n\n• ERP 송금현황 조회방법'),
 Document(id='9c24f7bd-af31-402b-b1b9-eaa9e1b189df', metadata={'images': ['014_img0.png'], 'origin_pdf': '무역관정산교육_v6_0923.pdf', 'page_num': 14.0, 'source': '014.md'}, page_content='2.3. 전도금배정현황\n============\n\n  \n\n2.3.1. ERP 메뉴 경로 : 전도관리 – 전도금배정/정산현황 - 전도금배정현황조회  \n2.3.2. 메뉴 설명 : 본사에서 송금된 상세내역을 기간별로 조회할수 있는 메뉴\n\n• ERP 전도금배정현황 조회방법\n=================='),
 Document(id='e3ae3298-64ee-470b-a9ca-38a6cb12a015', metadata={'images': ['010_img0.png'], 'origin_pdf': '무역관정산교육_v6_0923.pdf', 'page_num': 10.0, 'source': '010.md'}, page_content='2. 자금수령내역 확인 2.1. 전도자금출납명세서\n===========================\n\n  \n\n2.1.1. ERP 메뉴 경로 : 전도관리 – 전도금마감 – 전도자금출납명세서  \n2.1.2. ERP 메뉴 설명 : 본

In [46]:
pages = [int(doc.metadata["page_num"]) for doc in ai_message["source_documents"]]

In [47]:
pages

[7, 8]

In [26]:
sorted(set(pages)) 

[20, 21]

In [27]:
# 강의에서는 위처럼 진행하지만 업데이트된 LangChain 문법은 `.invoke()` 활용을 권장 
# 객체에서는 속성에 점(.)을 사용하여 접근
ai_message = qa_chain.invoke({"query": query})

# 답변 본문
answer = ai_message["result"]

# 모든 page_num 모으기
pages = [int(doc.metadata["page_num"]) for doc in ai_message["source_documents"]]
pages = sorted(set(pages))  # 중복 제거 + 정렬

origin = ai_message["source_documents"][0].metadata["origin_pdf"]  # 문서명 (1개라고 가정)

# 최종 출력
print(answer)
print(f"📖 출처: {origin} 페이지: {', '.join(map(str, pages))}")

자금집행 입력 내용을 수정하려면, K-ERP 전도관리의 '전도금집행관리(직접입력프로세스)' 메뉴로 이동하여 '집행내역 수정/삭제' 메뉴를 선택합니다. 수정하려는 내역을 선택한 후 '집행내역수정'을 클릭하여 수정하면 됩니다. 삭제하려면, 내역을 선택하고 '집행내역삭제'를 클릭하면 됩니다.
📖 출처: 무역관정산교육_v6_0923.pdf 페이지: 20, 21


In [45]:
# 어떤 문서가 검색되었는지 확인
retriever.invoke(query)

[Document(id='08d2c6a5-c4cb-4633-8d0f-08b81bb5cd3b', metadata={'images': [], 'origin_pdf': '무역관정산교육_v6_0923.pdf', 'page_num': 7.0, 'source': '007.md'}, page_content='1.2. 정산 기초 준비\n=============\n\n  \n\n1.2.1. 회계담당자 및 회계책임자 임명\n\n  \n\n회계책임자는 조직망의 장, 즉 관장님이 되십니다.\n\n  \n\n회계담당자는 1인무역관의 경우 관장님이 당연직 회계담당자가 되시고, 2인무역관의 경우 하위직급 직원이 당연직 회계담당자가  \n됩니다. 3인무역관의 경우 하위직급 직원중에서 지명하여 대 본사 보고(청렴서약서를 의미)를 해야합니다. BI, IT지원센터의 경우  \n회계담당자는 팀장님이 되십니다.(현지의 불가피한 사유로 부득이 다른 직원을 지명코자 할 경우 해당사유를 보고하여 재무팀 승인  \n필요)\n\n1.2.1.1. 회계담당자 및 회계책임자 책임\n\n  \n\n회계책 임자와 회계담당자는 회계관계직원 등의 책임에 관한 법률에 의한 법적 책임, 해외조직망 자금관리에 대한 책임을 집니다.\n\n[참고]\n\n  \n\n제4조(회계관계직원의 변상책임) ① 회계관계직원은 고의 또는 중대한 과실로 법령이나 그 밖의 관계 규정 및 예산에 정하여진 바를  \n위반하여 국가, 지방자치단체, 그 밖에 감사원의 감사를 받는 단체 등의 재산에 손해를 끼친 경우에는 변상할 책임이 있다.  \n② 현금 또는 물품을 출납·보관하는 회계관계직원은 선량한 관리자로서의 주의를 게을리하여 그가 보관하는 현금 또는 물품이  \n망실(亡失)되거나 훼손(毀損)된 경우에는 변상할 책임이 있다.\n\n· 무역관 인원별 회계담당자\n\n  \n\n| 1인 무역관 | 2인 무역관 | 3인 이상 무역관 |\n| --- | --- | --- |\n| 관장 | 하위직급 직원 | 하위직급 직원 중에서 지명하여 대 본사 보고 |'

## 5. Retrieval을 위한 keyword 사전 활용
- 직장인이라는 질의가 들어오면, 직장인을 거주자로 자동으로 바꾸도록 설정
- Knowledge Base에서 사용되는 keyword를 활용하여 사용자 질문 수정
- LangChain Expression Language (LCEL)을 활용한 Chain 연계

In [51]:
query = '4직급의 기준을 자세하게 알려주세요. 보기좋게 연번으로 알려주세요.' # 표를 마크다운으로 바꾸면 더 잘 읽는지 확인

In [52]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

dictionary = ["기준을 나타내는 표현 -> 경력 기준"]

prompt = ChatPromptTemplate.from_template(f"""
    사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
    만약 변경할 필요가 없다고 판단된다면, 사용자의 질문을 변경하지 않아도 됩니다.
    그런 경우에는 질문만 리턴해주세요.
    사전: {dictionary}
    
    질문: {{question}}
""")

dictionary_chain = prompt | llm | StrOutputParser()
tax_chain = {"query": dictionary_chain} | qa_chain

In [53]:
new_question = dictionary_chain.invoke({"question": query})

In [54]:
# 바뀐 질의
new_question

'4직급의 경력 기준을 자세하게 알려주세요. 보기 좋게 연번으로 알려주세요.'

In [26]:
ai_response = tax_chain.invoke({"question": query})

In [28]:
print(ai_response['result'])

4직급의 경력 기준은 다음과 같습니다:

1. 행정 분야에서 근무한 6급 공무원으로 5년 이상 경력 소지자
2. 정부투자기관, 경제단체 및 유관기관에서 동일직급 2년 이상 경력 소지자
3. 소령 2년 이상 경력 소지자
4. 민간기업 과장급으로 유관부문 2년 이상 경력 소지자
5. 대학 및 전문학교 전임강사 2년 이상 경력 소지자
6. 전문조사기관의 연구원으로 3년 이상 경력 소지자
7. 해당 분야에 실무경력, 연구 또는 연수 경력자로 당해 직급에 자격이 있다고 인사위원회에서 인정되는 자
8. 기타 전항과 동등한 자격이 있다고 인사위원회에서 인정되는 자
